In [ ]:
import pandas as pd
pd.set_option("max_colwidth", 100)
pd.set_option("display.max_rows",10)
pd.set_option("display.max_columns",5)
# DS= pd.read_csv('New_IMDB_Dataset.csv');
df= pd.DataFrame(DS,columns =['review', 'sentiment','LLM_sentiment','LLM_confidence']);
df


,review,sentiment,LLM_sentiment,LLM_confidence
0,one of the other reviewers has mentioned that after watching just 1 oz episode youll be hooked t...,positive,negative,80
1,a wonderful little production br br the filming technique is very unassuming very oldtimebbc fas...,positive,positive,95
2,i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air ...,positive,positive,95
3,basically theres a family where a little boy jake thinks theres a zombie in his closet his pare...,negative,negative,70
4,petter matteis love in the time of money is a visually stunning film to watch mr mattei offers u...,positive,positive,90
...,...,...,...,...
9995,fun entertaining movie about wwii german spy julie andrews falling in love with american pilot r...,positive,positive,80
9996,give me a break how can anyone say that this is a good hockey movie i know that movies tend to d...,negative,negative,90
9997,this movie is a bad movie but after watching an endless series of bad horror movies i can say th...,negative,negative,60
9998,this is a movie that was probably made to entertain the middle school early high school age kids...,negative,negative,80


In [ ]:
X=df["review"]
Y=df["sentiment"]
Y_LLM=df["LLM_sentiment"]
LLM_confidence=df["LLM_confidence"]

In [ ]:
import numpy as np
temp=np.where(LLM_confidence<=70)
temp=temp[0]

In [ ]:
df['LLM_FewShot'] = None

In [ ]:
newsetX=X[temp]
newsetY=Y[temp]
newsetYllm=Y_LLM[temp]
newsetCon=LLM_confidence[temp]

In [ ]:
!pip install openai

In [ ]:
import openai
openai.api_key  = ''

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
import time

def connect_with_api(prompt):
    max_retries = 3
    retries = 0

    while retries < max_retries:
        try:
            response = get_completion(prompt)
            return response
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            print(f"Retrying... (Attempt {retries} of {max_retries})")
            time.sleep(1)  # Wait for 1 second before retrying

    print("Failed to connect with the API after multiple attempts.")
    return None  # Return None or raise an exception, depending on your requirements

In [ ]:
i = 0  # Start with index 0 (This is for disconnections to API)

while i < len(newsetX):
  prompt = f"""
  Few-shot learning for Sentiment Analysis:

  For each of the following movie reviews, provide the sentiment as a single word(without other marks or words like 'sentimet:'), either "positive" or "negative".

  Use the few-shot learning examples below to improve your predictions:

  Few-shot Examples:
  '''{newsetX.iloc[0]} ''' sentiment:{newsetY.iloc[0]}
  '''{newsetX.iloc[1]} ''' sentiment:{newsetY.iloc[1]}
  '''{newsetX.iloc[2]} ''' sentiment:{newsetY.iloc[2]}
  Review text:
  '''{newsetX.iloc[i]}'''
  """
  response = connect_with_api(prompt)
  df.at[newsetX.index[i], 'LLM_FewShot'] = response
  print(i,' ',response)
  i+=1

In [ ]:
def clean_response(response_text):
    if response_text is None:
        return None
    elif "positive" in response_text:
        return "positive"
    elif "negative" in response_text:
        return "negative"
    else:
        return None

In [ ]:
df['LLM_FewShot']=df['LLM_FewShot'].apply(clean_response)

In [ ]:
df.to_csv('New_Few_IMDB_Dataset.csv', index=False)

In [ ]:
import pandas as pd
pd.set_option("max_colwidth", 100)
pd.set_option("display.max_rows",10)
pd.set_option("display.max_columns",5)
DS= pd.read_csv('New_fake_news.csv');
df= pd.DataFrame(DS,columns =['text', 'label','LLM_label','LLM_confidence','LLM_FewShot']);
df


In [ ]:
X=df["text"]
Y=df["label"]
Y_LLM=df["LLM_label"]
LLM_confidence=df["LLM_confidence"]
df["LLM_confidence"] = LLM_confidence.replace(0, 5)

In [ ]:
df['LLM_FewShot'] = None

In [ ]:
newsetX=X[temp]
newsetY=Y[temp]
newsetYllm=Y_LLM[temp]
newsetCon=LLM_confidence[temp]

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
import time

def connect_with_api(prompt):
    max_retries = 3
    retries = 0

    while retries < max_retries:
        try:
            response = get_completion(prompt)
            return response
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            print(f"Retrying... (Attempt {retries} of {max_retries})")
            time.sleep(1)  # Wait for 1 second before retrying

    print("Failed to connect with the API after multiple attempts.")
    return None  # Return None or raise an exception, depending on your requirements

In [ ]:
i = 0  # Start with index 0 (This is for disconnections to API)

while i < len(newsetX):
  prompt = f"""
  Determine the authenticity of the following news article:

  For the article provided, classify it as a single word (without other marks or words like 'authenticity:'), either "fake" or "real".

  Use the few-shot learning examples below to improve your prediction:

  Few-shot Examples:
  ```{newsetX.iloc[0]}``` authenticity:{newsetY.iloc[0]}
  ```{newsetX.iloc[1]}``` authenticity:{newsetY.iloc[1]}
  ```{newsetX.iloc[2]}``` authenticity:{newsetY.iloc[2]}
  Article text:
  ```{newsetX.iloc[i]}```
  """

  response = connect_with_api(prompt)
  df.at[newsetX.index[i], 'LLM_FewShot'] = response
  print(i,' ',response)
  i+=1

In [ ]:
def clean_response(response_text):
    if response_text is None:
        return None
    elif "real" in response_text:
        return "REAL"
    elif "fake" in response_text:
        return "FAKE"
    elif "REAL" in response_text:
        return "REAL"
    elif "FAKE" in response_text:
        return "FAKE"
    else:
        return None

In [ ]:
df.to_csv('New_Few_fake_news.csv', index=False)

In [ ]:
import pandas as pd
pd.set_option("max_colwidth", 100)
pd.set_option("display.max_rows",10)
pd.set_option("display.max_columns",5)
# DS= pd.read_pickle('New_movies_genre.pkl');
df= pd.DataFrame(DS,columns =['Plot', 'Genre','LLM_label','LLM_confidence']);
df


,Plot,Genre,LLM_label,LLM_confidence
0,alcoholic newspaperman steve bramley boards the ship san capador for a restful cruise hoping to ...,comedy,drama,80
1,jon arbuckle plans to propose to his girlfriend dr liz wilson who is going on a business trip to...,comedy,comedy,90
2,alexander villard is a former fencing champion who runs a highly competitive fencing school one ...,action,drama,90
3,in a mad max style future jake mcqueen is the ultimate smuggler smuggling in mexicans for money ...,action,action,90
4,the film is an ensemble comedydrama that focuses on the group of people each of whom is afraid o...,comedy,drama,90
...,...,...,...,...
3995,when eight year old oscar madly finds out that his parents are getting divorced he retreats into...,drama,drama,90
3996,a wealthy man by the name of ansford matt clark discovers his young wife laura lysette anthony h...,horror,horror,100
3997,young lawyer emily reed otis travels to new york city for an interview with an international law...,drama,drama,90
3998,the central figure of the film is represented by a mysterious figure called gaja gamini madhuri ...,drama,drama,90


In [ ]:
X=df["Plot"]
Y=df["Genre"]
Y_LLM=df["LLM_label"]
LLM_confidence=df["LLM_confidence"]

In [ ]:
import numpy as np
temp=np.where(LLM_confidence<=80)
temp=temp[0]

In [ ]:
df['LLM_FewShot'] = None

In [ ]:
newsetX=X[temp]
newsetY=Y[temp]
newsetYllm=Y_LLM[temp]
newsetCon=LLM_confidence[temp]

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
import time

def connect_with_api(prompt):
    max_retries = 3
    retries = 0

    while retries < max_retries:
        try:
            response = get_completion(prompt)
            return response
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            print(f"Retrying... (Attempt {retries} of {max_retries})")
            time.sleep(1)  # Wait for 1 second before retrying

    print("Failed to connect with the API after multiple attempts.")
    return None  # Return None or raise an exception, depending on your requirements

In [ ]:
i = 0  # Start with index 0 (This is for disconnections to API)

while i < len(newsetX):
  prompt = f"""
  Few-shot learning for Sentiment Analysis:

  For each of the following movie reviews, provide the sentiment as a single word(without other marks or words like 'sentimet:'), either "positive" or "negative".

  Use the few-shot learning examples below to improve your predictions:

  Few-shot Examples:
  '''{newsetX.iloc[0]} ''' sentiment:{newsetY.iloc[0]}
  '''{newsetX.iloc[1]} ''' sentiment:{newsetY.iloc[1]}
  '''{newsetX.iloc[2]} ''' sentiment:{newsetY.iloc[2]}
  Review text:
  '''{newsetX.iloc[i]}'''
  """
  prompt = f"""
  Determine the genre of the movie based on the following plot:

  For the plot provided, classify its genre as a single word (without other marks or words like 'genre:'), either "comedy", "action", "drama", or "horror".

  Use the few-shot learning examples below to improve your prediction:

  Few-shot Examples:
  ```{newsetX.iloc[0]}``` genre:{newsetY.iloc[0]}
  ```{newsetX.iloc[1]}``` genre:{newsetY.iloc[1]}
  ```{newsetX.iloc[2]}``` genre:{newsetY.iloc[2]}
  Movie plot:
  ```{newsetX.iloc[i]}```
  """

  response = connect_with_api(prompt)
  df.at[newsetX.index[i], 'LLM_FewShot'] = response
  print(i,' ',response)
  i+=1

In [ ]:
def clean_response(response_text):
    if response_text is None:
        return None
    elif "drama" in response_text:
        return "drama"
    elif "horror" in response_text:
        return "horror"
    elif "comedy" in response_text:
        return "comedy"
    elif "action" in response_text:
        return "action"
    else:
        return None

In [ ]:
df['LLM_FewShot']=df['LLM_FewShot'].apply(clean_response)

In [ ]:
df.to_pickle('New_Few_movies_genre.pkl')